In [19]:
import numpy as np
from functools import reduce
from scipy.linalg import eigh, null_space
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix
from scipy.sparse.linalg import eigsh, eigs
from qutip import Qobj, ptrace
from qutip import commutator as qt_commutator
from tqdm import tqdm
from itertools import product

In [20]:
#define numpy functions: commutator + partial trace

def commutator(A, B):
    return np.dot(A, B) - np.dot(B, A)

def partial_trace(rho, keep, dims):
    """Compute the partial trace of a density matrix."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

In [21]:
# 3 spins (0,1,2) H_pxp + SU(2) structure + Casimir operator

# Number of spins
N = 3

# Define Pauli matrices and identity
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
P = (1/2)*(I - Z)

# Construct Qx, Qy, Qz using Kronecker products
Q_x = (1/2) * sum((-1)**i * reduce(np.kron, [X if j == i else I for j in range(N)]) for i in range(N))
Q_y = (1/2) * sum((-1)**i * reduce(np.kron, [Y if j == i else I for j in range(N)]) for i in range(N))
Q_z = (1/2) * sum((-1)**i * reduce(np.kron, [Z if j == i else I for j in range(N)]) for i in range(N))

# Define the Casimir operator
Casimir = np.dot(Q_x, Q_x) + np.dot(Q_y, Q_y) + np.dot(Q_z, Q_z)

# Define the PXP Hamiltonian in the 3-spin restricted space
H_PXP = np.kron(np.kron(X, P), P) + np.kron(np.kron(P, X), P) + np.kron(np.kron(P, P), X)

In [22]:
# Check commutation relations

comm_x_y = commutator(Q_x, Q_y)
comm_y_z = commutator(Q_y, Q_z)
comm_z_x = commutator(Q_z, Q_x)
comm_c_x = commutator(Casimir, Q_x)
comm_c_y = commutator(Casimir, Q_y)
comm_c_z = commutator(Casimir, Q_z)

comm_H_PXP_c = commutator(H_PXP, Casimir)   # Verify that H_PXP commutes with Casimir


# Print results
print("[Q_x, Q_y] =\n", comm_x_y)
print("[Q_y, Q_z] =\n", comm_y_z)
print("[Q_z, Q_x] =\n", comm_z_x)
print("[Casimir, Q_x] =\n", comm_c_x)
print("[Casimir, Q_y] =\n", comm_c_y)
print("[Casimir, Q_z] =\n", comm_c_z)

print("[H_PXP, Casimir] =\n", comm_H_PXP_c)

# Commutation relations are satisfied - in particular H_PXP commutes with the Casimir operator in the 3-spin restricted space

[Q_x, Q_y] =
 [[0.+1.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.-0.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.-0.5j 0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.-0.5j 0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.-1.5j]]
[Q_y, Q_z] =
 [[0.+0.j  0.+0.5j 0.+0.5j 0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.5j 0.+0.j  0.+0.j  0.+0.5j 0.+0.j  0.+0.5j 0.+0.j  0.+0.j ]
 [0.+0.5j 0.+0.j  0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.5j 0.+0.j ]
 [0.+0.j  0.+0.5j 0.+0.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.5j]
 [0.+0.5j 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.5j 0.+0.5j 0.+0.j ]
 [0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.5j 0.+0.j  0.+0.j  0.+0.5j]
 [0.+0.j  0.+0.j  0.+0.5j 0.+0.j 

In [23]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def pxp(n_spins, pbc=True, sparse=True):
    """
    Constructs the Hamiltonian for the PXP model.
    
    Parameters:
        N (int): Number of spins.
        pbc (bool): Whether to use periodic boundary conditions.
        sparse (bool): Whether to return a sparse matrix.
    
    Returns:
        H (scipy.sparse.csr_matrix or np.ndarray): The Hamiltonian matrix.
    """
    # Identity matrix
    I = identity(2, format="csr") if sparse else np.eye(2)
    
    # Pauli matrices
    X = csr_matrix(pauli_x()) if sparse else pauli_x()
    Z = csr_matrix(pauli_z()) if sparse else pauli_z()
    P = csr_matrix(0.5 * (I - Z)) if sparse else 0.5 * (I - Z)
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**n_spins, 2**n_spins), dtype=np.float64) if sparse else np.zeros((2**n_spins, 2**n_spins), dtype=np.float64)
    
    # Build the Hamiltonian for open boundary conditions
    for i in range(1, n_spins-1):
        term = 1
        for j in range(n_spins):
            if j == i:
                term = kron(term, X, format="csr") if sparse else np.kron(term, X)
            elif j == i - 1 or j == i + 1:
                term = kron(term, P, format="csr") if sparse else np.kron(term, P)
            else:
                term = kron(term, I, format="csr") if sparse else np.kron(term, I)
        H += term

    # Add PBC terms if enabled
    if pbc:
        # First PBC term: coupling between the last and the first site - P_N+1 = P_1
        term1 = 1
        for j in range(n_spins):
            if j == 0:
                term1 = kron(term1, X, format="csr") if sparse else np.kron(term1, X)
            elif j == 1 or j == n_spins - 1:
                term1 = kron(term1, P, format="csr") if sparse else np.kron(term1, P)
            else:
                term1 = kron(term1, I, format="csr") if sparse else np.kron(term1, I)
        H += term1

        # Second PBC term: coupling between the first and the last site - P_0 = P_N
        term2 = 1
        for j in range(n_spins):
            if j == 0 or j == n_spins - 2:
                term2 = kron(term2, P, format="csr") if sparse else np.kron(term2, P)
            elif j == n_spins - 1:
                term2 = kron(term2, X, format="csr") if sparse else np.kron(term2, X)
            else:
                term2 = kron(term2, I, format="csr") if sparse else np.kron(term2, I)
        H += term2

    return H

def g_state():
    """Returns the |g> state."""
    return np.array([[0], [1]])

def r_state():
    """Returns the |r> state."""
    return np.array([[1], [0]])

def neel_state(n_spins):  # allowed only by OBC if n_spins is odd
    """
    Constructs the Neel state for a given number of spins.
    
    Parameters:
        n_spins (int): Number of spins in the system.
    
    Returns:
        np.ndarray: The Neel state as a Kronecker product of |r> and |g> states.
    """
    state = r_state() if n_spins % 2 == 1 else g_state()
    for i in range(1, n_spins):
        if i % 2 == 0:
            state = np.kron(state, r_state())
        else:
            state = np.kron(state, g_state())
    return state

def antineel_state(n_spins): # allowed by both OBC and PBC
    """
    Constructs the anti-Neel state for a given number of spins.
    
    Parameters:
        n_spins (int): Number of spins in the system.
    
    Returns:
        np.ndarray: The anti-Neel state as a Kronecker product of |r> and |g> states.
    """
    state = g_state() if n_spins % 2 == 1 else r_state()
    for i in range(1, n_spins):
        if i % 2 == 0:
            state = np.kron(state, g_state())
        else:
            state = np.kron(state, r_state())
    return state

# define first the allowed states by rydberg blockade and then the PXP Hamiltonian in this subspace
def construct_pxp_hamiltonian(n_spins, pbc=True):
    """
    Constructs the PXP Hamiltonian matrix for a system of n_spins with specified boundary conditions,
    enforcing the Rydberg blockade constraint.
    
    Parameters:
        n_spins (int): Number of spins in the system.
        boundary_condition (str): "OBC" for open boundary conditions or "PBC" for periodic boundary conditions.

    Returns:
        H (numpy.ndarray): The PXP Hamiltonian matrix in the constrained subspace.
        basis (list): The list of allowed basis states in binary format.
    """
    # Generate all possible states in the full Hilbert space
    full_basis = list(product([0, 1], repeat=n_spins))
    
    # Filter states to respect the Rydberg blockade constraint
    allowed_basis = []
    for state in full_basis:
        if pbc: # PBC
            # Check that no two adjacent 1's exist, including across periodic boundaries
            if all(state[i] + state[(i + 1) % n_spins] <= 1 for i in range(n_spins)):
                allowed_basis.append(state)
        else:  # OBC
            # Check that no two adjacent 1's exist
            if all(state[i] + state[i + 1] <= 1 for i in range(n_spins - 1)):
                allowed_basis.append(state)
    
    # Map basis states to indices
    basis_size = len(allowed_basis)
    state_to_index = {state: idx for idx, state in enumerate(allowed_basis)}
    
    # Construct the full PXP Hamiltonian
    full_H = pxp(n_spins, pbc= True, sparse=False)
    
    # Project the full Hamiltonian onto the subspace defined by the allowed basis states
    H = np.zeros((basis_size, basis_size), dtype=float)
    for i, state_i in enumerate(allowed_basis):
        for j, state_j in enumerate(allowed_basis):
            idx_i = int("".join(map(str, state_i)), 2)
            idx_j = int("".join(map(str, state_j)), 2)
            H[i, j] = full_H[idx_i, idx_j]
    
    return H, allowed_basis

In [24]:
# 8 spins total system, 3 spins subsystem - I want to find H_pxp in the whole system, its ground state - which is scarred, its density matrix, and then find the 3-spins rdm by tracing out the 5 spins

# Number of spins
n_spins = 8

# Construct the Hamiltonian
H = pxp(n_spins, pbc=True, sparse=True)

# ground state of H only + debugging partial trace

# Compute smallest eigenvalue and the corresponding eigenvector
min_eigenvalue, min_eigenvector = eigsh(H, k=1, which='SA')

print(f"Minimum eigenvalue: {min_eigenvalue}")
print(f"Corresponding eigenvector shape: {min_eigenvector.shape}")
#print(f"Corresponding eigenvector: {min_eigenvector}")

#Construct the density matrix
density_matrix = np.outer(min_eigenvector, min_eigenvector.conj())

print(f"Density matrix shape: {density_matrix.shape}")
#print(f"Density matrix: {density_matrix}")

Minimum eigenvalue: [-4.83095867]
Corresponding eigenvector shape: (256, 1)
Density matrix shape: (256, 256)


In [25]:
# use qutip for partial trace

density_matrix_qobj = Qobj(density_matrix, dims=[[2]*n_spins, [2]*n_spins])
#density_matrix_qobj = Qobj(density_matrix, dims=[2**n_spins, 2**n_spins])
print(f"qobj density matrix shape: {density_matrix_qobj.shape}")
#print(f"qobj density matrix: {density_matrix_qobj}")

# Trace out some qubits (keep other qubits)
keep_qubits = [0,1,2]  # Indices of qubits to keep
traced_out_density_matrix = ptrace(density_matrix_qobj, keep_qubits)

# Convert the result back to a dense matrix if needed
traced_out_density_matrix_dense = traced_out_density_matrix.full()

print(type(traced_out_density_matrix))
print(type(traced_out_density_matrix_dense))

print(f"Traced out density matrix shape: {traced_out_density_matrix.shape}")
print(f"Traced out dense density matrix shape: {traced_out_density_matrix_dense.shape}")

print(f"Traced out density matrix: {traced_out_density_matrix}")
print(f"Traced out dense density matrix: {traced_out_density_matrix_dense}")

# Diagonalize the traced out density matrix
eigenvalues, eigenvectors = np.linalg.eigh(traced_out_density_matrix_dense)
#print(f"Eigenvalues of the traced out density matrix: {eigenvalues}")
#print(f"Eigenvectors of the traced out density matrix: {eigenvectors}")

# Print the minimum eigenvalue
min_eigenvalue = np.min(eigenvalues)
print(f"Minimum eigenvalue: {min_eigenvalue}")

qobj density matrix shape: (256, 256)
<class 'qutip.core.qobj.Qobj'>
<class 'numpy.ndarray'>
Traced out density matrix shape: (8, 8)
Traced out dense density matrix shape: (8, 8)
Traced out density matrix: Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 2.56906354e-33  2.04562191e-34  3.19336115e-18 -6.23729147e-18
   8.35518206e-34  9.26017225e-20  1.95821526e-18  6.08450773e-19]
 [ 2.04562191e-34  3.73616077e-33 -9.27926725e-19  6.00344046e-18
  -6.61884155e-34  1.28596076e-17  4.34564479e-18 -1.81017460e-17]
 [ 3.19336115e-18 -9.27926725e-19  5.92947927e-02 -8.05178745e-02
  -1.31671478e-17 -7.49454027e-02 -8.05178745e-02  1.27203196e-01]
 [-6.23729147e-18  6.00344046e-18 -8.05178745e-02  1.37543542e-01
   1.65835268e-17  1.35970726e-01  1.09646882e-01 -2.21417043e-01]
 [ 8.35518206e-34 -6.61884155e-34 -1.31671478e-17  1.65835268e-17
   5.84817165e-33  1.77493541e-17  2.25743123e-17 -3.05108832e-17]
 [ 9.26017225e-20  1

In [26]:
# use your function for partial trace

# Trace out some qubits (keep other qubits)
dims = [2]*n_spins
keep_qubits = [0,1,2]  # Indices of qubits to keep
traced_out_density_matrix = partial_trace(density_matrix, keep_qubits, dims)

print(type(traced_out_density_matrix))

print(f"Traced out density matrix shape: {traced_out_density_matrix.shape}")

print(f"Traced out density matrix: {traced_out_density_matrix}")

# Diagonalize the traced out density matrix
eigenvalues, eigenvectors = np.linalg.eigh(traced_out_density_matrix)
#print(f"Eigenvalues of the traced out density matrix: {eigenvalues}")
#print(f"Eigenvectors of the traced out density matrix: {eigenvectors}")

# Print the minimum eigenvalue
min_eigenvalue = np.min(eigenvalues)
print(f"Minimum eigenvalue: {min_eigenvalue}")

<class 'numpy.ndarray'>
Traced out density matrix shape: (8, 8)
Traced out density matrix: [[ 2.56906354e-33  2.04562191e-34  3.19336115e-18 -6.23729147e-18
   8.35518206e-34  9.26017225e-20  1.95821526e-18  6.08450773e-19]
 [ 2.04562191e-34  3.73616077e-33 -9.27926725e-19  6.00344046e-18
  -6.61884155e-34  1.28596076e-17  4.34564479e-18 -1.81017460e-17]
 [ 3.19336115e-18 -9.27926725e-19  5.92947927e-02 -8.05178745e-02
  -1.31671478e-17 -7.49454027e-02 -8.05178745e-02  1.27203196e-01]
 [-6.23729147e-18  6.00344046e-18 -8.05178745e-02  1.37543542e-01
   1.65835268e-17  1.35970726e-01  1.09646882e-01 -2.21417043e-01]
 [ 8.35518206e-34 -6.61884155e-34 -1.31671478e-17  1.65835268e-17
   5.84817165e-33  1.77493541e-17  2.25743123e-17 -3.05108832e-17]
 [ 9.26017225e-20  1.28596076e-17 -7.49454027e-02  1.35970726e-01
   1.77493541e-17  1.96838335e-01  1.35970726e-01 -3.01934917e-01]
 [ 1.95821526e-18  4.34564479e-18 -8.05178745e-02  1.09646882e-01
   2.25743123e-17  1.35970726e-01  1.37543542

In [27]:
#  check if 3-spins rdm commutes with the 3-spins Casimir operator - all np operators

commutator_rdm = commutator(traced_out_density_matrix, Casimir)
print("[rho_rdm, Casimir] =\n", commutator_rdm)
commutator_rdm_qobj = commutator(traced_out_density_matrix_dense, Casimir)
print("[rho_rdm, Casimir] =\n", commutator_rdm_qobj)

[rho_rdm, Casimir] =
 [[ 0.00000000e+00+0.j -3.19336115e-18+0.j  6.38672230e-18+0.j
   1.86561354e-18+0.j -3.19336115e-18+0.j  4.46427966e-18+0.j
  -6.32989320e-18+0.j  0.00000000e+00+0.j]
 [ 3.19336115e-18+0.j  0.00000000e+00+0.j  5.92947927e-02+0.j
  -8.05178745e-02+0.j -1.22392211e-17+0.j -7.49454027e-02+0.j
  -8.05178745e-02+0.j  1.27203196e-01+0.j]
 [-6.38672230e-18+0.j -5.92947927e-02+0.j  0.00000000e+00+0.j
   1.55463277e-01+0.j -5.92947927e-02+0.j  1.61035749e-01+0.j
   1.55463277e-01+0.j -2.54406392e-01+0.j]
 [-1.86561354e-18+0.j  8.05178745e-02+0.j -1.55463277e-01+0.j
   0.00000000e+00+0.j  8.05178745e-02+0.j  8.56186373e-02+0.j
   2.77555756e-17+0.j -8.05178745e-02+0.j]
 [ 3.19336115e-18+0.j  1.22392211e-17+0.j  5.92947927e-02+0.j
  -8.05178745e-02+0.j  0.00000000e+00+0.j -7.49454027e-02+0.j
  -8.05178745e-02+0.j  1.27203196e-01+0.j]
 [-4.46427966e-18+0.j  7.49454027e-02+0.j -1.61035749e-01+0.j
  -8.56186373e-02+0.j  7.49454027e-02+0.j  0.00000000e+00+0.j
  -8.56186373e-02+0

In [28]:
# Define a small threshold based on machine precision
threshold = np.finfo(float).eps

# Function to set entries smaller than machine precision to zero
def set_small_entries_to_zero(matrix, threshold):
    matrix[np.abs(matrix) < threshold] = 0
    return matrix

# Set small entries to zero
commutator_rdm = set_small_entries_to_zero(commutator_rdm, threshold)
commutator_rdm_qobj = set_small_entries_to_zero(commutator_rdm_qobj, threshold)

# Print the commutator
print("[rho_rdm, Casimir] =\n", commutator_rdm)
print("[rho_rdm, Casimir] =\n", commutator_rdm_qobj)

[rho_rdm, Casimir] =
 [[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.05929479+0.j -0.08051787+0.j
   0.        +0.j -0.0749454 +0.j -0.08051787+0.j  0.1272032 +0.j]
 [ 0.        +0.j -0.05929479+0.j  0.        +0.j  0.15546328+0.j
  -0.05929479+0.j  0.16103575+0.j  0.15546328+0.j -0.25440639+0.j]
 [ 0.        +0.j  0.08051787+0.j -0.15546328+0.j  0.        +0.j
   0.08051787+0.j  0.08561864+0.j  0.        +0.j -0.08051787+0.j]
 [ 0.        +0.j  0.        +0.j  0.05929479+0.j -0.08051787+0.j
   0.        +0.j -0.0749454 +0.j -0.08051787+0.j  0.1272032 +0.j]
 [ 0.        +0.j  0.0749454 +0.j -0.16103575+0.j -0.08561864+0.j
   0.0749454 +0.j  0.        +0.j -0.08561864+0.j  0.16103575+0.j]
 [ 0.        +0.j  0.08051787+0.j -0.15546328+0.j  0.        +0.j
   0.08051787+0.j  0.08561864+0.j  0.        +0.j -0.08051787+0.j]
 [ 0.        +0.j -0.1272032 +0.j  0.25440639+0

In [29]:
def commutes_with(matrix, candidate):
    """Check if two matrices commute."""
    return np.allclose(np.dot(matrix, candidate), np.dot(candidate, matrix))

print("Do Casimir and rdm commute?", commutes_with(Casimir, traced_out_density_matrix))
print("Do Casimir and H_pxp commute for 3 spins?", commutes_with(Casimir, H_PXP))

Do Casimir and rdm commute? False
Do Casimir and H_pxp commute for 3 spins? True


In [30]:
# find all possible 8x8 matrices that commute with the 3 spins rdm

# Define a random 8x8 matrix A
np.random.seed(42)  # For reproducibility
A = np.random.randint(-10, 10, (8, 8))

# Define the 64x64 commutation matrix C
C = np.zeros((64, 64))

# Fill C such that it encodes the equation AX - XA = 0
for i in range(8):
    for j in range(8):
        E_ij = np.zeros((8, 8))
        E_ij[i, j] = 1  # Basis matrix E_ij
        commutator = A @ E_ij - E_ij @ A  # Compute [A, E_ij]
        C[:, i * 8 + j] = commutator.flatten()  # Store in C

# Compute the null space (solutions to C * x = 0)
null_C = null_space(C)

# Reshape the basis vectors of the null space into 8x8 matrices
commuting_matrices = [null_C[:, i].reshape(8, 8) for i in range(null_C.shape[1])]


In [31]:
# now do it for the 3 spins rdm

# Define the 64x64 commutation matrix C
C = np.zeros((64, 64))

# Fill C such that it encodes the equation AX - XA = 0
for i in range(8):
    for j in range(8):
        E_ij = np.zeros((8, 8))
        E_ij[i, j] = 1  # Basis matrix E_ij
        commutator = traced_out_density_matrix_dense @ E_ij - E_ij @ traced_out_density_matrix_dense  # Compute [A, E_ij]
        C[:, i * 8 + j] = commutator.flatten()  # Store in C

# Compute the null space (solutions to C * x = 0)
null_C = null_space(C)

# Reshape the basis vectors of the null space into 8x8 matrices
commuting_matrices = [null_C[:, i].reshape(8, 8) for i in range(null_C.shape[1])]

C:\Users\camipolv\AppData\Local\Temp\ipykernel_13736\3653864305.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  C[:, i * 8 + j] = commutator.flatten()  # Store in C


In [32]:
print(f"Number of commuting matrices: {len(commuting_matrices)}")
print(f"Example of a commuting matrix:\n{commuting_matrices[0]}")

Number of commuting matrices: 14
Example of a commuting matrix:
[[ 2.21290164e-02  9.06736837e-01  1.85855705e-14  6.05229963e-14
  -2.65506932e-01  2.95201142e-13  6.37157852e-14  2.43883953e-13]
 [ 2.13788092e-01 -1.19302619e-01  1.70952175e-14  7.78806925e-14
   1.26335009e-01  4.00857333e-13  8.05705602e-14  3.28492165e-13]
 [ 1.30728761e-13 -3.99376121e-14  3.10133821e-04 -2.71324902e-02
   1.89527568e-14  1.02552632e-02 -2.71324902e-02  1.02479582e-02]
 [ 4.12586632e-13 -1.49264063e-13 -2.71324902e-02  3.16161702e-02
   9.53679668e-14  2.64564563e-02  6.55366621e-03 -7.40202191e-03]
 [ 1.29874087e-02  2.83037485e-04 -1.01922665e-14 -3.81964091e-14
   7.32023796e-04 -1.86991478e-13 -3.82255482e-14 -1.53877291e-13]
 [ 1.96548333e-12 -7.31073751e-13  1.02552632e-02  2.64564563e-02
   4.85334480e-13  1.00319738e-01  2.64564563e-02  2.11753744e-02]
 [ 4.16972012e-13 -1.49946270e-13 -2.71324902e-02  6.55366621e-03
   9.54255398e-14  2.64564563e-02  3.16161702e-02 -7.40202191e-03]
 [ 1.

In [33]:
def find_commuting_matrix(A):
    """Find a matrix that commutes with the given 8x8 matrix A."""
    n = A.shape[0]
    assert n == 8, "The input matrix must be 8x8."

    # Create the commutation matrix C
    C = np.zeros((n**2, n**2), dtype=np.complex128)
    
    for i in range(n):
        for j in range(n):
            Eij = np.zeros((n, n), dtype=np.complex128)
            Eij[i, j] = 1
            commutator = np.dot(A, Eij) - np.dot(Eij, A)
            C[:, i * n + j] = commutator.flatten()  # Store in C

    # Find the null space of the commutation matrix C
    null_space_C = null_space(C)
    
    # Reshape the null space vectors to form the commuting matrices
    commuting_matrices = [null_space_C[:, i].reshape((n, n)) for i in range(null_space_C.shape[1])]
    
    return commuting_matrices

In [34]:
commuting_matrices = find_commuting_matrix(traced_out_density_matrix_dense)

print(f"Found {len(commuting_matrices)} matrices that commute with 3 spins rdm.")
for i, matrix in enumerate(commuting_matrices):
    print(f"Commuting matrix {i+1}:\n", matrix)

Found 14 matrices that commute with 3 spins rdm.
Commuting matrix 1:
 [[ 2.26053978e-02-0.j -9.46856658e-01-0.j -1.62483126e-13-0.j
  -5.03130799e-13-0.j -1.09227261e-01-0.j -2.38467227e-12-0.j
  -5.05062546e-13-0.j -1.96790087e-12-0.j]
 [ 9.99648645e-03-0.j -7.37315182e-02-0.j -2.81616483e-14-0.j
  -1.26431940e-13-0.j -7.44876632e-02-0.j -6.38859129e-13-0.j
  -1.26257484e-13-0.j -5.23252409e-13-0.j]
 [ 6.80007726e-15-0.j -2.98575764e-14-0.j  3.49831891e-02-0.j
  -5.65968714e-02-0.j  4.87211183e-14-0.j  3.28067806e-02-0.j
  -5.65968714e-02-0.j -1.11922080e-03-0.j]
 [ 3.80034427e-14-0.j -9.00117068e-14-0.j -5.65968714e-02-0.j
   6.88905305e-02-0.j  1.85372380e-13-0.j  2.14321146e-02-0.j
   2.12278421e-02-0.j  9.42411201e-04-0.j]
 [ 8.29279092e-03-0.j  4.41403396e-03-0.j  3.49780423e-15-0.j
   6.80366890e-15-0.j -4.45849788e-04-0.j  2.76616317e-14-0.j
   6.70879024e-15-0.j  2.33992940e-14-0.j]
 [ 1.97036831e-13-0.j -4.19239296e-13-0.j  3.28067806e-02-0.j
   2.14321146e-02-0.j  9.14567033

In [35]:
# check how distant these commuting matrices are from the 3 spins casimir operator 

def frobenius_norm(A, B):
    """Compute the Frobenius norm of the difference between two matrices."""
    return np.linalg.norm(A - B, 'fro')

# Assuming find_commuting_matrix and traced_out_density_matrix_dense are defined
commuting_matrices = find_commuting_matrix(traced_out_density_matrix_dense)

print(f"Found {len(commuting_matrices)} matrices that commute with 3 spins rdm.")
for i, matrix in enumerate(commuting_matrices):
    frobenius_distance = frobenius_norm(matrix, Casimir)
    #print(f"Commuting matrix {i+1}:\n", matrix)
    print(f"Frobenius norm with Casimir matrix: {frobenius_distance}\n")

Found 14 matrices that commute with 3 spins rdm.
Frobenius norm with Casimir matrix: 7.5773374888120095

Frobenius norm with Casimir matrix: 7.9461831526482865

Frobenius norm with Casimir matrix: 7.903475802544961

Frobenius norm with Casimir matrix: 7.190688356716495

Frobenius norm with Casimir matrix: 7.588306287624998

Frobenius norm with Casimir matrix: 8.131934991247164

Frobenius norm with Casimir matrix: 7.466760635689004

Frobenius norm with Casimir matrix: 7.754872357450279

Frobenius norm with Casimir matrix: 7.651412945827589

Frobenius norm with Casimir matrix: 7.824161604311004

Frobenius norm with Casimir matrix: 7.833287440668953

Frobenius norm with Casimir matrix: 7.517229324003762

Frobenius norm with Casimir matrix: 8.034619983410712

Frobenius norm with Casimir matrix: 7.486402748216963



In [ ]:
# find other possible commuting candidates for the 3 spins rdm starting from the symmetries of the pxp modeltion, spatial inversion, particle-hole symmmetry - restricted to the 3 spins subspace